In [2]:
import sys
sys.executable

'/usr/bin/python'

In [14]:
import os

In [58]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize, sent_tokenize 
stop_words = set(stopwords.words('english')) 
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/skullscript/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/skullscript/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/skullscript/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/skullscript/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [62]:
from sklearn.datasets import load_files
import re

movie_data = load_files(r"./review_polarity/txt_sentoken")
x, y = movie_data.data, movie_data.target
#print(x[0])

documents = []

stemmer = WordNetLemmatizer()
#preprocessing the data getting rid of numbers special characters etc

for sent in range(0, len(x)):
    #removing special character
    document = re.sub(r'\W', ' ', str(x[sent]))
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    document = re.sub(r'^b\s+', '', document)
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    #print(document)
    documents.append(document)
    #print("------------------------\n")


In [60]:
# converting text to number using the bag of words model
from sklearn.feature_extraction.text import CountVectorizer
# max feature 1500 min_numDoc = 5 contating feature, only include those word that occur max of 70% of all doc
# word that occurs in every doc is not suitable for classification does not provide any unique information about doc
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
x = vectorizer.fit_transform(documents).toarray()

[0 0 0 ... 0 0 0]


In [65]:
# term frequency
# bag of words drawback! It doesn't take avvount the fact that the word might also be having high frequency occurent
# in other documents as well 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
x = tfidfconverter.fit_transform(documents).toarray()

TypeError: __init__() got an unexpected keyword argument 'min_def'